In [1]:
import os
import tifffile
from mm import *

### Washing block
Washing tubes with PBS -- Do this when starting a new type of experiment.

- Put the tubings to be used in the PBS containing beaker/bottle.
- Run this block before starting experiment.


In [2]:
loadSample('b2', 1)
loadSample('s1', 1)
loadSample('s2', 1)
loadSample('s3', 1)
loadSample('s4', 1)

loadSample('b4', 1)
loadSample('s5', 1)
loadSample('s6', 1)
loadSample('s7', 1)
loadSample('s8', 1)

loadSample('b5', 1)
loadSample('s9', 1)
loadSample('s10', 1)
loadSample('s11', 1)

### Calibration

- Connect all tubes to the tube containing GH-AF-480 probe (50mM) & buffer tubes to their respective buffers.
- Run this block which will take the timelapse of the GH-AF-480 probe (50nM) through the FCS chamber.
- Copy the printed time in a file.
- Open each port timelapse images in ImageT. Go to Image>Stacks>Plot Z-axis Profile.
- It will plot an intensity change graph. Click on the List button on the bottom left of the graph.
- Save these values in an excel sheet. Repeat for all the ports. 
- Choose the suitable peak time when the intensity reached maximum in the FCS2 chamber.
- Note the number for each prot and mulitply it with 8.3 (which is the flow rate ul per second).
- This will be the tube_vols for each port.


In [ ]:
def take_time_lapse(savefile, n_rounds):
    mmc.set_exposure(500)
    selectChannel('FL-565')
    time.sleep(2.5)
    with tifffile.TiffWriter(savefile) as tif:
        startPump()
        t0 = time.time()
        time_points = np.zeros((n_rounds,))
        for round in range(n_rounds):
            time_points[round] = time.time() - t0
            img = takeImage(False)
            tif.write(img, contiguous = True)
            time.sleep(0.4)
    stopPump()
    selectChannel('DIC')
    return time_points

all_ports = [('b2','b1'), ('s1','b1'), ('s2','b1'), ('s3','b1'), ('s4','b1'), 
             ('b4','b3'), ('s5','b3'), ('s6','b3'), ('s7','b3'), ('s8','b3'), 
             ('s9','b5'), ('s10','b5'), ('s11','b5')]

for port, buffer_port in all_ports:
    print("Port:" + port)
    loadSample(port, 0.2)
    loadSample(buffer_port, 0.001)
    time_points = take_time_lapse('calibration-' + port + '.tif', 180)
    print(time_points)
    #loadSample(buffer_port, 0.5)

### Initialization Step
This step load samples into the initial segment of the tubing
- First insert tubings into the correct sample vessels
- List all sample ports used in exp in 'sample_ports'
- List all buffer ports used in exp in 'buffer_ports'
- run this block (shift_enter)

In [ ]:
sample_ports = ['s1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11']
buffer_ports = ['b5', 'b2', 'b1', 'b3', 'b4']

for port in sample_ports:
    loadSample(port, 0.2)

for port in buffer_ports:
    loadSample(port, 1)

### Here the real experiment start

CHECKLIST:
    
    - in Micromanager, check that the Changroup is selected as 'Channel'
    - in Micromanager, check that the Autofocus setting is selected as 'Duo'
    - in Micromanager, make sure the stage position list is open and positions are selected. Save the positions.
    - Make sure the bulb is on.

In [7]:
stainings = ['s9', 's9', 's9', 's9', 's9', 's9', 's9', 's9', 's9', 's9']
washing = ['b5', 'b5', 'b5', 'b5', 'b5', 'b5', 'b5', 'b5', 'b5', 'b5']
mmc.set_exposure(500)
idx = 1
for st, wa in zip(stainings[(idx-1):], washing[(idx-1):]):
    #staining
    loadIncubateWash(st, 60, wa)
    imagingStep(pathPrefix = 's-Round-%i'%idx, channels=['DIC','FL-647'], grid=(5,5))
    
    #Protease
    loadIncubateWash('s10', 60, 'b5')
    imagingStep(pathPrefix = 'p-Round-%i'%idx, channels=['DIC','FL-647'], grid=(5,5))
    
    #NaBH4
    loadSample('b2', 2.5)
    loadSample('b1', 2)
    imagingStep(pathPrefix = 'b-Round-%i'%idx, channels=['DIC','FL-647'], grid=(5,5))
    
    idx += 1

### Test

In [3]:
imagingStep(pathPrefix = 'test', channels=['DIC', 'FL-647'], grid=(3,3))

In [6]:
loadSample('b5', 1)

In [4]:
sample_ports = ['s9', 's10', 's11']
buffer_ports = ['b5', 'b5', 'b5']

for port in sample_ports:
    loadSample(port, 0.2)

for port in buffer_ports:
    loadSample(port, 0.4)